# പ്രോംപ്റ്റ് എഞ്ചിനീയറിങ്ങിലേക്ക് പരിചയം
പ്രോംപ്റ്റ് എഞ്ചിനീയറിങ് എന്നത് സ്വാഭാവിക ഭാഷാ പ്രോസസ്സിംഗ് പ്രവർത്തനങ്ങൾക്ക് പ്രോംപ്റ്റുകൾ രൂപകൽപ്പന ചെയ്യുകയും മെച്ചപ്പെടുത്തുകയും ചെയ്യാനുള്ള പ്രക്രിയയാണ്. ശരിയായ പ്രോംപ്റ്റുകൾ തിരഞ്ഞെടുക്കൽ, അവയുടെ പാരാമീറ്ററുകൾ ട്യൂൺ ചെയ്യൽ, അവയുടെ പ്രകടനം വിലയിരുത്തൽ എന്നിവ ഇതിൽ ഉൾപ്പെടുന്നു. NLP മോഡലുകളിൽ ഉയർന്ന കൃത്യതയും കാര്യക്ഷമതയും നേടുന്നതിന് പ്രോംപ്റ്റ് എഞ്ചിനീയറിങ് അത്യന്താപേക്ഷിതമാണ്. ഈ വിഭാഗത്തിൽ, OpenAI മോഡലുകൾ ഉപയോഗിച്ച് പ്രോംംപ്റ്റ് എഞ്ചിനീയറിങ്ങിന്റെ അടിസ്ഥാനങ്ങൾ പരിശോധിക്കും.


### Exercise 1: Tokenization
OpenAI-യിൽ നിന്നുള്ള ഒരു ഓപ്പൺ-സോഴ്‌സ് ഫാസ്റ്റ് ടോക്കനൈസറായ tiktoken ഉപയോഗിച്ച് ടോക്കനൈസേഷൻ അന്വേഷിക്കുക
കൂടുതൽ ഉദാഹരണങ്ങൾക്ക് [OpenAI Cookbook](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb?WT.mc_id=academic-105485-koreyst) കാണുക.


In [ ]:
# EXERCISE:
# 1. Run the exercise as is first
# 2. Change the text to any prompt input you want to use & re-run to see tokens

import tiktoken

# Define the prompt you want tokenized
text = f"""
Jupiter is the fifth planet from the Sun and the \
largest in the Solar System. It is a gas giant with \
a mass one-thousandth that of the Sun, but two-and-a-half \
times that of all the other planets in the Solar System combined. \
Jupiter is one of the brightest objects visible to the naked eye \
in the night sky, and has been known to ancient civilizations since \
before recorded history. It is named after the Roman god Jupiter.[19] \
When viewed from Earth, Jupiter can be bright enough for its reflected \
light to cast visible shadows,[20] and is on average the third-brightest \
natural object in the night sky after the Moon and Venus.
"""

# Set the model you want encoding for
encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")

# Encode the text - gives you the tokens in integer form
tokens = encoding.encode(text)
print(tokens);

# Decode the integers to see what the text versions look like
[encoding.decode_single_token_bytes(token) for token in tokens]

### Exercise 2: Github മോഡലുകളുടെ കീ സെറ്റപ്പ് സാധൂകരിക്കുക

താഴെ കൊടുത്തിരിക്കുന്ന കോഡ് റൺ ചെയ്ത് നിങ്ങളുടെ Github മോഡലുകളുടെ എന്റ്പോയിന്റ് ശരിയായി സെറ്റ് ചെയ്തിട്ടുണ്ടോ എന്ന് പരിശോധിക്കുക. കോഡ് ഒരു ലളിതമായ അടിസ്ഥാന പ്രോംപ്റ്റ് പരീക്ഷിച്ച് പൂർത്തീകരണം സാധൂകരിക്കുന്നു. `oh say can you see` എന്ന ഇൻപുട്ട് `by the dawn's early light..` എന്ന തരത്തിൽ പൂർത്തിയാകണം.


In [2]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential

token = os.environ["GITHUB_TOKEN"]
endpoint = "https://models.inference.ai.azure.com"

model_name = "gpt-4o"

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

def get_completion(prompt, client, model_name, temperature=1.0, max_tokens=1000, top_p=1.0):
    response = client.complete(
        messages=[
            {
                "role": "system",
                "content": "You are a helpful assistant.",
            },
            {
                "role": "user",
                "content": prompt,
            },
        ],
        model=model_name,
        temperature=temperature,
        max_tokens=max_tokens,
        top_p=top_p
    )
    return response.choices[0].message.content

## ---------- Call the helper method

### 1. Set primary content or prompt text
text = f"""
oh say can you see
"""

### 2. Use that in the prompt template below
prompt = f"""
```{text}```
"""

## 3. Run the prompt
response = get_completion(prompt, client, model_name)
print(response)


That line is the opening lyric of "The Star-Spangled Banner," the national anthem of the United States, written by Francis Scott Key. If you'd like more information or analysis, feel free to ask!


### Exercise 3: കൃത്രിമ നിർമ്മിതികൾ  
ഒരു വിഷയത്തെക്കുറിച്ച് പ്രോംപ്റ്റ് നൽകിയപ്പോൾ അത് നിലവിലില്ലാത്തതായിരിക്കാം, അല്ലെങ്കിൽ അത് മുൻപരിചയപ്പെട്ട ഡാറ്റാസെറ്റിന്റെ പുറത്തുള്ള (കൂടുതൽ പുതിയ) വിഷയങ്ങളായിരിക്കാം, അപ്പോൾ LLM പൂർത്തീകരണങ്ങൾ നൽകുമ്പോൾ എന്ത് സംഭവിക്കുന്നു എന്ന് പരിശോധിക്കുക. വ്യത്യസ്ത പ്രോംപ്റ്റ് പരീക്ഷിച്ചാൽ അല്ലെങ്കിൽ വ്യത്യസ്ത മോഡൽ ഉപയോഗിച്ചാൽ പ്രതികരണം എങ്ങനെ മാറുന്നു എന്ന് കാണുക.


In [ ]:

## Set the text for simple prompt or primary content
## Prompt shows a template format with text in it - add cues, commands etc if needed
## Run the completion 
text = f"""
generate a lesson plan on the Martian War of 2076.
"""

prompt = f"""
```{text}```
"""

response = get_completion(prompt, client, model_name)
print(response)

### Exercise 4: Instruction Based 
"text" വേരിയബിൾ പ്രാഥമിക ഉള്ളടക്കമായി സജ്ജമാക്കാൻ ഉപയോഗിക്കുക  
"prompt" വേരിയബിൾ ആ പ്രാഥമിക ഉള്ളടക്കവുമായി ബന്ധപ്പെട്ട ഒരു നിർദ്ദേശം നൽകാൻ ഉപയോഗിക്കുക.

ഇവിടെ നാം മോഡലിനോട് ആ ടെക്സ്റ്റ് രണ്ടാം ക്ലാസ് വിദ്യാർത്ഥിക്ക് സംക്ഷേപിക്കാൻ ആവശ്യപ്പെടുന്നു.


In [4]:
# Test Example
# https://platform.openai.com/playground/p/default-summarize

## Example text
text = f"""
Jupiter is the fifth planet from the Sun and the \
largest in the Solar System. It is a gas giant with \
a mass one-thousandth that of the Sun, but two-and-a-half \
times that of all the other planets in the Solar System combined. \
Jupiter is one of the brightest objects visible to the naked eye \
in the night sky, and has been known to ancient civilizations since \
before recorded history. It is named after the Roman god Jupiter.[19] \
When viewed from Earth, Jupiter can be bright enough for its reflected \
light to cast visible shadows,[20] and is on average the third-brightest \
natural object in the night sky after the Moon and Venus.
"""

## Set the prompt
prompt = f"""
Summarize content you are provided with for a second-grade student.
```{text}```
"""

## Run the prompt
response = get_completion(prompt, client, model_name)
print(response)

Jupiter is the fifth planet from the Sun and the biggest one in our Solar System. It's made of gas and is much bigger than all the other planets put together! You can see Jupiter in the night sky because it's very bright. People have noticed it for a really long time and named it after a Roman god.


### വ്യായാമം 5: സങ്കീർണ്ണ പ്രോംപ്റ്റ്  
സിസ്റ്റം, ഉപയോക്താവ്, അസിസ്റ്റന്റ് സന്ദേശങ്ങൾ ഉള്ള ഒരു അഭ്യർത്ഥന ശ്രമിക്കുക  
സിസ്റ്റം അസിസ്റ്റന്റിന്റെ പശ്ചാത്തലം സജ്ജമാക്കുന്നു  
ഉപയോക്താവ് & അസിസ്റ്റന്റ് സന്ദേശങ്ങൾ ബഹുസ്ഥായിയായ സംഭാഷണ പശ്ചാത്തലം നൽകുന്നു  

സിസ്റ്റം പശ്ചാത്തലത്തിൽ അസിസ്റ്റന്റിന്റെ വ്യക്തിത്വം "വ്യംഗ്യപരമായ" ആയി സജ്ജമാക്കിയിരിക്കുന്നതിനെ ശ്രദ്ധിക്കുക.  
മറ്റൊരു വ്യക്തിത്വ പശ്ചാത്തലം ഉപയോഗിച്ച് ശ്രമിക്കുക. അല്ലെങ്കിൽ മറ്റൊരു ഇൻപുട്ട്/ഔട്ട്പുട്ട് സന്ദേശങ്ങളുടെ പരമ്പര ശ്രമിക്കുക.


In [5]:
response = client.complete(
    model=model_name,
    messages=[
        {"role": "system", "content": "You are a sarcastic assistant."},
        {"role": "user", "content": "Who won the world series in 2020?"},
        {"role": "assistant", "content": "Who do you think won? The Los Angeles Dodgers of course."},
        {"role": "user", "content": "Where was it played?"}
    ]
)
print(response.choices[0].message.content)

Oh, you mean the famous 2020 World Series that wasn’t in a regular location? That was the year they played in the glamorous Arlington, Texas, at Globe Life Field.


### Exercise: നിങ്ങളുടെ അന്തർദൃഷ്ടി അന്വേഷിക്കുക
മുകളിൽ നൽകിയ ഉദാഹരണങ്ങൾ നിങ്ങൾക്ക് പുതിയ പ്രോംപ്റ്റുകൾ (സാധാരണ, സങ്കീർണ്ണം, നിർദ്ദേശം തുടങ്ങിയവ) സൃഷ്ടിക്കാൻ ഉപയോഗിക്കാവുന്ന മാതൃകകൾ നൽകുന്നു - ഉദാഹരണങ്ങൾ, സൂചനകൾ എന്നിവ പോലുള്ള നാം ചർച്ച ചെയ്ത മറ്റ് ആശയങ്ങൾ അന്വേഷിക്കാൻ മറ്റ് വ്യായാമങ്ങൾ സൃഷ്ടിക്കാൻ ശ്രമിക്കുക.


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**അസൂയാ**:  
ഈ രേഖ AI വിവർത്തന സേവനം [Co-op Translator](https://github.com/Azure/co-op-translator) ഉപയോഗിച്ച് വിവർത്തനം ചെയ്തതാണ്. നാം കൃത്യതയ്ക്ക് ശ്രമിച്ചിട്ടുണ്ടെങ്കിലും, സ്വയം പ്രവർത്തിക്കുന്ന വിവർത്തനങ്ങളിൽ പിശകുകൾ അല്ലെങ്കിൽ തെറ്റുകൾ ഉണ്ടാകാമെന്ന് ദയവായി ശ്രദ്ധിക്കുക. അതിന്റെ മാതൃഭാഷയിലുള്ള യഥാർത്ഥ രേഖ അധികാരപരമായ ഉറവിടമായി കണക്കാക്കപ്പെടണം. നിർണായക വിവരങ്ങൾക്ക്, പ്രൊഫഷണൽ മനുഷ്യ വിവർത്തനം ശുപാർശ ചെയ്യപ്പെടുന്നു. ഈ വിവർത്തനത്തിന്റെ ഉപയോഗത്തിൽ നിന്നുണ്ടാകുന്ന ഏതെങ്കിലും തെറ്റിദ്ധാരണകൾക്കോ തെറ്റായ വ്യാഖ്യാനങ്ങൾക്കോ ഞങ്ങൾ ഉത്തരവാദികളല്ല.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
